In [12]:
import pandas as pd
from os.path import join, splitext

specifications_path = join(output_folder, 'specifications.xlsx')
json_output_path = join(output_folder, 'assets.json')

excel_writer = pd.ExcelWriter(specifications_path)
for file_name in [
    'specs_data.csv',
    'scenario_info.csv',
    'scenario_parameters.csv',
]:
    table = pd.read_csv(join(input_folder, file_name))
    base_name = splitext(file_name)[0]
    sheet_name = ''.join(_.title() for _ in base_name.split('_'))
    table.to_excel(excel_writer, sheet_name=sheet_name, index=False)
excel_writer.save()

In [13]:
from os import makedirs
from os.path import join

details_folder = join(output_folder, 'details')
summaries_folder = join(output_folder, 'summaries')
try:
    makedirs(details_folder)
    makedirs(summaries_folder)
except OSError:
    pass

In [14]:
from onsset.runner import calibration, scenario

cities_path = join(input_folder, 'cities.csv')
calibrated_specifications_path = join(output_folder, 'specifications.xlsx')
calibrated_cities_path = join(output_folder, 'cities.csv') 

calibration(
    specs_path=specifications_path,
    csv_path=cities_path,
    specs_path_calib=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path)
scenario(
    specs_path=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path,
    results_folder=details_folder,
    summary_folder=summaries_folder)

2021-07-14 15:34:07,423		Populate ResidentialDemandTier columns
2021-07-14 15:34:07,425		Ensure that columns that are supposed to be numeric are numeric
2021-07-14 15:34:07,442		Replace null values with zero
2021-07-14 15:34:07,444		Sort by country, Y and X
2021-07-14 15:34:07,447		Classify road dist
2021-07-14 15:34:07,450		Classify substation dist
2021-07-14 15:34:07,456		Classify elevation
2021-07-14 15:34:07,464		Classify slope
2021-07-14 15:34:07,468		Classify land cover
2021-07-14 15:34:07,471		Combined classification
2021-07-14 15:34:07,475		Grid penalty
2021-07-14 15:34:07,479		Calculate Wind CF
2021-07-14 15:34:07,532		Population calibration process
2021-07-14 15:34:07,583		Urban/rural calibration process
2021-07-14 15:34:07,716		Population projection process


The calibrated population differs by 0.00. In case this is not acceptable please revise this part of the code


2021-07-14 15:34:07,926		Calibrate current electrification
2021-07-14 15:34:08,030		Calibration finished. Results are transferred to the csv file


We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration
The modelled electrification rate differ by 0.00. Urban elec. rate differ by 0.00 and Rural elec. rate differ by 0.00. 
If this is not acceptable please revise this part of the algorithm


2021-07-14 15:34:08,267		Determine current MV line length
2021-07-14 15:34:08,274		Calculate new connections
2021-07-14 15:34:08,285		Setting electrification demand as per target per year
2021-07-14 15:34:08,339		Calculate minigrid hydro LCOE


...

2021-07-14 15:34:08,401		Calculate minigrid PV LCOE
2021-07-14 15:34:08,444		Calculate minigrid wind LCOE
2021-07-14 15:34:08,481		Calculate standalone PV LCOE
2021-07-14 15:34:08,508		Determine minimum technology (off-grid)
2021-07-14 15:34:08,537		Determine minimum tech LCOE
2021-07-14 15:34:08,550		Define the initial electrification status
2021-07-14 15:34:08,659		Initially 245 electrified
2021-07-14 15:34:08,741		Electrification loop 1 with 245 electrified
2021-07-14 15:34:08,835		Calculate investment cost
2021-07-14 15:34:08,842		Determine electrification limits
2021-07-14 15:34:08,853		Determine final electrification decision
2021-07-14 15:34:08,856		Calculate new capacity
2021-07-14 15:34:08,875		Calculate summaries
2021-07-14 15:34:08,934		Calculate new connections
2021-07-14 15:34:08,944		Setting electrification demand as per target per year
2021-07-14 15:34:08,960		Calculate minigrid hydro LCOE
2021-07-14 15:34:09,015		Calculate minigrid PV LCOE


The electrification rate achieved in 2025 is 83.3 %


2021-07-14 15:34:09,060		Calculate minigrid wind LCOE
2021-07-14 15:34:09,116		Calculate standalone PV LCOE
2021-07-14 15:34:09,143		Determine minimum technology (off-grid)
2021-07-14 15:34:09,169		Determine minimum tech LCOE
2021-07-14 15:34:09,181		Define the initial electrification status
2021-07-14 15:34:09,287		Initially 245 electrified
2021-07-14 15:34:09,378		Electrification loop 1 with 453 electrified
2021-07-14 15:34:09,452		Electrification loop 2 with 49 electrified
2021-07-14 15:34:09,518		Electrification loop 3 with 8 electrified
2021-07-14 15:34:09,576		Electrification loop 4 with 1 electrified
2021-07-14 15:34:09,608		Calculate investment cost
2021-07-14 15:34:09,624		Determine electrification limits
2021-07-14 15:34:09,632		Determine final electrification decision
2021-07-14 15:34:09,638		Calculate new capacity
2021-07-14 15:34:09,658		Calculate summaries


The electrification rate achieved in 2030 is 100.0 %


2021-07-14 15:34:09,944		Finished


In [15]:
from glob import glob
from os.path import basename, splitext

def collect_scenarios(target_path, source_folder):
    tables = []
    for path in glob(join(source_folder, '*.csv')):
        table = pd.read_csv(path)
        table['ScenarioName'] = basename(splitext(path)[0])
        columns = list(table.columns)
        columns.insert(0, columns.pop(columns.index('ScenarioName')))
        tables.append(table[columns])
    pd.concat(tables).to_csv(target_path, index=False)
    
collect_scenarios(join(output_folder, 'details.csv'), details_folder)
collect_scenarios(join(output_folder, 'summaries.csv'), summaries_folder)

In [16]:
calculated_data_path = join(output_folder, 'details.csv')

base_electricity_structure = {
    'assets': {
        'assetById': {},
        'assetTypeByCode': {},
        'assetsGeoJson': {},
        'styles': {}
    }
}

table = pd.read_csv(calculated_data_path)

summary = table.describe()

reserved_cols = {'X_deg','Y_deg', 'Country', 'id', 'Elevation', 'ScenarioName', }

columns = [*set(table.columns) - reserved_cols]
table['id'] = table['ScenarioName'] + '_' + table['id'].astype(str)
table.set_index('id')

columns

['Slope',
 'PerCapitaDemand',
 'MGDieselFuelCost2025',
 'MVConnectDist',
 'MinimumOverallLCOE2025',
 'ElecStatusIn2025',
 'CurrentMVLineDist',
 'Pop2030',
 'SA_PV2030',
 'RoadDist',
 'MG_Hydro2025',
 'MG_PV2025',
 'IsUrban',
 'Minimum_Tech_Off_grid2030',
 'GridDistCalibElec',
 'EnergyPerSettlement2025',
 'EducationDemand',
 'MinGridDist2030',
 'PopStartYear',
 'NumPeoplePerHH',
 'Minimum_LCOE_Off_grid2030',
 'InvestmentCapita2025',
 'ResidentialDemandTier2',
 'ResidentialDemandTier3',
 'HydropowerDist',
 'WindCF',
 'CommercialDemand',
 'MinimumOverallLCOE2030',
 'Pop2025Low',
 'MinimumOverall2025',
 'Pop2025',
 'ResidentialDemandTier4',
 'ElectrificationOrder2025',
 'FinalElecCode2025',
 'minTDdist',
 'Pop2025High',
 'NewCapacity2030',
 'EnergyPerSettlement2030',
 'MGDieselFuelCost2030',
 'Pop',
 'ElecStart',
 'HealthDemand',
 'ResidentialDemandTierCustom',
 'LandCover',
 'FinalElecCode2018',
 'PlannedHVLineDist',
 'InvestmentCost2025',
 'TravelHours',
 'Off_Grid_Code2025',
 'Residenti

In [17]:
def quantile_strategie(column, row, summary):
    try:
        if row[column] == summary[column]['min']:
            return 'min'
        elif row[column] <= summary[column]['25%']:
            return '25%'
        elif row[column] <= summary[column]['50%']:
            return '50%'
        elif row[column] <= summary[column]['75%']:
            return '75%'
        else:
            return 'max'
    except KeyError:
        return row[column]

def get_name(row):
    return f'{row["Country"]} {row["id"].split("_")[-1]}'

def generate_assets_by_id(df):
    assets_by_id = {}
    for index, row in df.iterrows():
        assets_by_id[row['id']] = {
            "name": get_name(row),
            "typeCode": "m",
            "utilityId": "",
            "connections": {},
            'attributes': row[[*columns]].to_dict(),
        }
        
    return assets_by_id

def generate_assets_geojson(df):
    features = [{
        "geometry": {
            "type": "Point",
            "coordinates": [
              row['X_deg'],
              row['Y_deg'],
            ]
        },
        "type": "Feature",
        "properties": {
            "id": row['id'],
            "typeCode": "m",
            'styles': {
                TOOL_ID: {column: quantile_strategie(column, row, summary) for column in columns}
            }
        }
    } for index, row in df.iterrows()]
    
    return {
      "type": "FeatureCollection",
      "features": features
    }

assets_geojson = generate_assets_geojson(table.head(100))
assets_by_id =  generate_assets_by_id(table.head(100))


#summary['Minimum_Tech_Off_grid2030']
#assets_by_id

In [18]:
gray_color = [128, 128, 128, 160]
blue_color = [3, 169, 244, 160]
green_color = [92, 176, 81, 160]
yellow_color = [255, 255, 0, 160]
red_color = [255, 0, 0, 160]

inner_style_dictionary = {
    'min': {
        'fillColor': gray_color,
        'lineColor': gray_color,
        'lineWidth': 5
        # 'dashArray': [1, 1]
    },
    '25%': {
        'fillColor': blue_color,
        'lineColor': blue_color,
        'lineWidth': 10
        # 'dashArray': [1, 1]
    },
    '50%': {
        'fillColor': green_color,
        'lineColor': green_color,
        'lineWidth': 15
        # 'dashArray': [1, 1]
    },
    '75%': {
        'fillColor': red_color,
        'lineColor': red_color,
        'lineWidth': 20
        # 'dashArray': [1, 1]
    },
    'max': {
        'fillColor': red_color,
        'lineColor': red_color,
        'lineWidth': 20
        # 'dashArray': [1, 1]
    },
}


categorical_columns = list(table.select_dtypes(include=['object']).columns)
general_styles = { column: inner_style_dictionary for column in columns if column not in categorical_columns }
general_styles['CommercialDemand']


{'min': {'fillColor': [128, 128, 128, 160],
  'lineColor': [128, 128, 128, 160],
  'lineWidth': 5},
 '25%': {'fillColor': [3, 169, 244, 160],
  'lineColor': [3, 169, 244, 160],
  'lineWidth': 10},
 '50%': {'fillColor': [92, 176, 81, 160],
  'lineColor': [92, 176, 81, 160],
  'lineWidth': 15},
 '75%': {'fillColor': [255, 0, 0, 160],
  'lineColor': [255, 0, 0, 160],
  'lineWidth': 20},
 'max': {'fillColor': [255, 0, 0, 160],
  'lineColor': [255, 0, 0, 160],
  'lineWidth': 20}}

In [19]:
def get_type(column):
    if column not in categorical_columns:
        return column, 'number'
    
    return column, 'string'
    
asset_types = {
    'm': {
        'assetAttributes': [
          get_type(column)  for column in columns
        ]
    }
}


asset_types

{'m': {'assetAttributes': [('Slope', 'number'),
   ('PerCapitaDemand', 'number'),
   ('MGDieselFuelCost2025', 'number'),
   ('MVConnectDist', 'number'),
   ('MinimumOverallLCOE2025', 'number'),
   ('ElecStatusIn2025', 'number'),
   ('CurrentMVLineDist', 'number'),
   ('Pop2030', 'number'),
   ('SA_PV2030', 'number'),
   ('RoadDist', 'number'),
   ('MG_Hydro2025', 'number'),
   ('MG_PV2025', 'number'),
   ('IsUrban', 'number'),
   ('Minimum_Tech_Off_grid2030', 'string'),
   ('GridDistCalibElec', 'number'),
   ('EnergyPerSettlement2025', 'number'),
   ('EducationDemand', 'number'),
   ('MinGridDist2030', 'number'),
   ('PopStartYear', 'number'),
   ('NumPeoplePerHH', 'number'),
   ('Minimum_LCOE_Off_grid2030', 'number'),
   ('InvestmentCapita2025', 'number'),
   ('ResidentialDemandTier2', 'number'),
   ('ResidentialDemandTier3', 'number'),
   ('HydropowerDist', 'number'),
   ('WindCF', 'number'),
   ('CommercialDemand', 'number'),
   ('MinimumOverallLCOE2030', 'number'),
   ('Pop2025Low'

In [20]:
import json

base_electricity_structure['assets']['assetTypeByCode'] = asset_types
base_electricity_structure['assets']['assetsGeoJson'] = assets_geojson
base_electricity_structure['assets']['assetById'] = assets_by_id
base_electricity_structure['assets']['styles'][TOOL_ID] = general_styles

json.dump(base_electricity_structure, open(json_output_path, 'wt'), indent=4)